In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

import numpy.testing as npt

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement,
    get_y_measurement,
    get_z_measurement
)
from quara.objects.gate import (
    Gate, get_depolarizing_channel,
    get_x_rotation,
    get_amplitutde_damping_channel
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [10]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Tester Objects (State)
# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_states = [state_x0, state_y0, state_z0, state_z1]

In [11]:
tester_povms = [get_x_measurement(c_sys), get_y_measurement(c_sys), get_z_measurement(c_sys)]

In [16]:
# True Object
true_objects = []
true_objects.append(get_depolarizing_channel(p=0, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=0.05, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=1, c_sys=c_sys))
true_objects.append(get_x_rotation(theta=np.pi/2, c_sys=c_sys))
true_objects.append(get_x_rotation(theta=np.pi, c_sys=c_sys))
true_objects.append(get_amplitutde_damping_channel(gamma=0.1, c_sys=c_sys))

true_object = true_objects[0]

In [17]:
num_data = [100, 1000]
n_rep = 100

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardQpt(
        tester_states, tester_povms, on_para_eq_constraint=True, seed=seed
    ),
    StandardQpt(
        tester_states, tester_povms, on_para_eq_constraint=False, seed=seed
    ),
    StandardQpt(
        tester_states, tester_povms, on_para_eq_constraint=True, seed=seed
    ),
    StandardQpt(
        tester_states, tester_povms, on_para_eq_constraint=False, seed=seed
    ),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []
elapsed_times = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 0/100 [00:00<?, ?it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LinearEstimator


  2%|▏         | 2/100 [00:00<00:07, 12.30it/s]

elapsed_time:0.13486006657282512[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/gate.py:430: ComplexWarning: Casting complex values to real discards the imaginary part
  hs[alpha, beta] = np.trace(np.kron(b_alpha, b_beta) @ choi)


elapsed_time:0.12970934708913168[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]

elapsed_time:0.9415506005287171[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator


100%|██████████| 100/100 [00:51<00:00,  1.93it/s]

elapsed_time:0.864634080727895[min]



In [24]:
qtomo_list = [e_list[0].qtomography for e_list in estimation_results_list]
list(set(qtomo_list))

In [25]:
qtomo_list = [e_list[0].qtomography for e_list in estimation_results_list]
qtomo_type_dict = {}

for qtomo in qtomo_list:
    qtomo_type = (qtomo.__class__, qtomo.on_para_eq_constraint)
    qtomo_type_dict[qtomo_type] = qtomo

In [27]:
len(qtomo_type_dict)

2

In [26]:
for item in qtomo_type_dict.items():
    print(item)

((<class 'quara.protocol.qtomography.standard.standard_qpt.StandardQpt'>, True), <quara.protocol.qtomography.standard.standard_qpt.StandardQpt object at 0x11297deb0>)
((<class 'quara.protocol.qtomography.standard.standard_qpt.StandardQpt'>, False), <quara.protocol.qtomography.standard.standard_qpt.StandardQpt object at 0x11297d310>)


In [30]:
data_analysis.make_mses_graph_estimation_results(
    estimation_results_list, case_name_list, true_object, show_analytical_results=True
)

In [10]:
data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_name_list,
    true_object,
    tester_objects=dict(states=tester_states, povms=tester_povms),
)

In [ ]:
"""
report.export_report("sample_povmt_case02_軸から一番離れた表面_nrep=1000.pdf",
    estimation_results_list,
    case_name_list,
    qtomography_list,
    para_list,
    estimator_list,
    true_object,
    tester_objects,
    seed=seed,
    computation_time=sum(elapsed_times)
)
""""""